In [ ]:
import random
import copy
import networkx as nx
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import math
import pickle
import os
import time
from datetime import datetime
from scipy.stats import qmc
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
random.seed(datetime.now().timestamp())

In [ ]:
!unzip /content/gdrive/MyDrive/Networks/test_cases.zip -d /

In [ ]:
num_of_nodes = [20, 25, 30, 35, 40, 45, 50, 55, 60]
test_case_num = 30

#defining network parameters
GATEWAY_ID = 0
spread_factor = range(7,13)

In [ ]:
#initializing particle population (priority and velocity vector of each particle) randomly
number_particles = 30
init_angle = (-180, 180)
init_angle_velocity = (-5,5)
init_priority = (-1000,1000)
init_velocity = (-100,100)
max_iterations = 500
large_negative_value = -10**6
large_positive_value = 10**6
C = 0.5
alpha = 1
beta = 2
c1 = 2.05
c2 = 2.05
phi = c1 + c2
constriction_factor = 2/abs(2-phi-math.sqrt(phi**2-4*phi))

In [ ]:
class Particle:
  def __init__(self,angle, sf_priority, angle_velocity, sf_velocity ,fitness):
    self.angle = angle
    self.sf_priority = sf_priority
    self.angle_velocity = angle_velocity
    self.sf_velocity = sf_velocity
    self.fitness = fitness

  def __str__(self):
    return f'''
               angle          = {self.angle}\n
               sf_priority    = {self.sf_priority}\n
               angle_velocity = {self.angle_velocity}\n
               sf_velocity    = {self.sf_velocity}\n
               fitness        = {self.fitness}\n
            '''

In [ ]:
def dist(a,b):
  x1,y1 = a
  x2,y2 = b
  return math.sqrt((x2-x1)**2+(y2-y1)**2)

In [ ]:
def read_data(path, isGraph):
  if isGraph:
    G = nx.read_gml(path,destringizer=int)
    return G
  with open(path,'rb') as file:
    data = pickle.load(file)
    file.close()
    return data


In [ ]:
def decode_path(G, edge_nodes, particle_position):
  N_DEVICES = len(G.nodes)
  ans = {}
  for f in edge_nodes:
    visited_nodes = [False for i in range(N_DEVICES)]
    iteration = 0
    terminal_node, sf = (f, max(particle_position.sf_priority[f], key = lambda x: particle_position.sf_priority[f][x]))
    path = []
    while terminal_node != GATEWAY_ID and iteration <= N_DEVICES:
      visited_nodes[terminal_node] = True
      iteration = iteration + 1
      closest_node = -1
      smallest_dist = large_positive_value
      for adj_node_id in G.adj[terminal_node]:
        if visited_nodes[adj_node_id]:
          continue
        
        #get adjecent node coordinates
        x0,y0 = G.nodes()[adj_node_id]['pos']

        #get terminal node coordinates
        Px,Py = G.nodes()[terminal_node]['pos']

        #get terminal node angle (convert to radians)
        theta = math.radians(particle_position.angle[terminal_node])

        #calculate distance
        distance = abs(math.cos(theta)*(Py-y0)-math.sin(theta)*(Px-x0))

        #find the closest point
        if distance < smallest_dist:
          smallest_dist = distance
          closest_node = adj_node_id
        
      if closest_node == -1:
        break

      next_node, next_sf = (closest_node, max(particle_position.sf_priority[closest_node], key = lambda x: particle_position.sf_priority[closest_node][x]))
      
      path.append((terminal_node, next_node, sf))
      terminal_node = next_node
      sf = next_sf
    ans[f] = path
  return ans
      

In [ ]:
def calculate_fitness(particle_position, iteration, edge_nodes, G, network_data):
  fitness_cost = 0
  path = decode_path(G, edge_nodes, particle_position)
  for f in edge_nodes:
    #adding a penalty if invalid path is returned
    if len(path[f]) == 0 or path[f][-1][1] != GATEWAY_ID:
      fitness_cost = fitness_cost + large_positive_value
      continue

    #Calculate cost of path and adding link capacity penalty
    total_delay = 0
    for i,j,k in path[f]:
      fitness_cost = fitness_cost + network_data['cst'][f,i,j,k]
      if network_data['max_edge_data_rate'][k]-network_data['data_rate'][f,i,j,k] < 0:
        fitness_cost = fitness_cost + network_data['cst'][f,i,j,k] + ((C*iteration)**alpha)*((network_data['max_edge_data_rate'][k]-network_data['data_rate'][f,i,j,k])**beta) 
      total_delay = total_delay + network_data['edge_delays'][f,i,j] + network_data['tx_time'][k] + network_data['eh_time'][i,k]      

    #Adding delay constraint penalty
    if network_data['max_flow_delays'][f]-total_delay < 0:
      fitness_cost = fitness_cost + ((C*iteration)**alpha)*((network_data['max_flow_delays'][f]-total_delay)**beta)
  
  particle_position.fitness = fitness_cost

In [ ]:
def node_angle_values(G):
  queue = [GATEWAY_ID]
  N_DEIVCES = len(G.nodes)
  visited = [False for node in range(N_DEVICES)]
  visited[GATEWAY_ID] = True
  angle_values = {}
  angle_values[GATEWAY_ID] = 0
  while len(queue) != 0:
    for i in range(len(queue)):
      front = queue.pop(0)
      x1,y1 = G.nodes[front]['pos']
      for adj_nodes in G.adj[front]:
        if not visited[adj_nodes]:
          visited[adj_nodes] = True
          x2,y2 = G.nodes[adj_nodes]['pos']
          if x1 == x2:
            angle_values[adj_nodes] = 90.0
          else:
            angle_values[adj_nodes] = math.degrees(math.atan((y2-y1)/(x2-x1)))
          queue.append(adj_nodes)
  return angle_values

In [ ]:
def heur_init(G):
  N_DEVICES = len(G.nodes)
  particle = []
  angle_values = node_angle_values(G)
  return [Particle(
      {u: angle_values[u] + random.randint(-5,5) for u in range(N_DEVICES)},
      {u: {k : random.randint(*init_priority) for k in spread_factor} for u in range(N_DEVICES)},
      {u: 0 for u in range(N_DEVICES)},
      {u: {k : 0 for k in spread_factor} for u in range(N_DEVICES)},
      0
  ) for p in range(math.ceil(number_particles*0.1))]

In [ ]:
def PSO_Algorithm(edge_nodes, G, network_data):
  start = time.time()
  end = time.time()
  converge_iter = 0
  N_DEVICES = len(G.nodes)

  #particle initialization (Halton)
  sampler = qmc.Halton(d = N_DEVICES, scramble=False)
  sampler1 = qmc.Halton(d = 6, scramble=False)
  angles = sampler.integers(l_bounds=-180, u_bounds=180, n = math.floor(number_particles*0.9), endpoint = True).tolist()
  priorities = sampler1.integers(l_bounds = -1000, u_bounds = 1000, n = math.floor(number_particles*0.9)*N_DEVICES, endpoint = True).tolist()

  particle = [Particle(
      angles[p],
      [{spread_factor[k]:priorities[p*N_DEVICES+u][k] for k in range(6)} for u in range(N_DEVICES)],
      {u : 0 for u in range(N_DEVICES)},
      {u : {k : 0 for k in spread_factor} for u in range(N_DEVICES)},
      0
  ) for p in range(math.floor(number_particles*0.9))]

  #Particle initialization (heuristic initialization)
  particle = particle + heur_init(G)

  pBest = copy.deepcopy(particle)   #initially each particle is best
  gBest = copy.deepcopy(particle[0]) #initially assume first particle as global best 

  calculate_fitness(gBest, 0, edge_nodes, G, network_data)

  #calculate fitness of each particle and its neighbour
  for p in range(number_particles):
    calculate_fitness(particle[p],0,edge_nodes, G, network_data)
    calculate_fitness(pBest[p], 0, edge_nodes, G, network_data)

  #PSO Algorithm
  for iteration in range(1,max_iterations+1):

    #update pBest for each particle
    for p in range(number_particles):
      if particle[p].fitness < pBest[p].fitness:
        pBest[p] = copy.deepcopy(particle[p])

    #finding the global best
    for p in range(number_particles):
      if pBest[p].fitness < gBest.fitness:
        gBest = copy.deepcopy(pBest[p])
        converge_iter = iteration
        end = time.time()

    #calculate velocity
    for p in range(number_particles):
      for u in range(N_DEVICES):
        r1, r2 = np.random.uniform(low = 0,high = 1,size = 2).tolist()
        particle[p].angle_velocity[u] = round(constriction_factor*(particle[p].angle_velocity[u] + c1*r1*(pBest[p].angle[u]-particle[p].angle[u]) + c2*r2*(gBest.angle[u]-particle[p].angle[u])))
        for k in spread_factor:
          r1, r2 = np.random.uniform(low = 0,high = 1,size = 2).tolist()
          particle[p].sf_velocity[u][k] = round(constriction_factor*(particle[p].sf_velocity[u][k] + c1*r1*(pBest[p].sf_priority[u][k]-particle[p].sf_priority[u][k]) + c2*r2*(gBest.sf_priority[u][k]-particle[p].sf_priority[u][k])))

    #Update Positions
    for p in range(number_particles):
      for u in range(N_DEVICES):
        particle[p].angle[u] = particle[p].angle[u] + particle[p].angle_velocity[u]
        for k in spread_factor:
          particle[p].sf_priority[u][k] = particle[p].sf_priority[u][k] + particle[p].sf_velocity[u][k]

    #Calculate fitness of all particles
    for p in range(number_particles):
      calculate_fitness(particle[p],iteration,edge_nodes, G, network_data)

  #decode solution
  ans = decode_path(G, edge_nodes, gBest)

  return [gBest.fitness, ans, converge_iter, (end-start)]

In [ ]:
expected_ans = read_data("/content/gdrive/MyDrive/Networks/expected_ans.pkl", False)

In [ ]:
route_optimality_ratio = {}
near_route_optimality_ratio = {}
route_failure_ratio = {}
near_route_failure_ratio = {}
results = {}
percent_gap = {}
iterations_to_converge = {}
time_to_converge = {}
invalid_tests = {}
for N_DEVICES in num_of_nodes:
  correct_solution = 0
  nearly_correct = 0
  total_tests = 0
  invalid_graph = 0
  results[N_DEVICES] = []
  percent_gap[N_DEVICES] = []
  iterations_to_converge[N_DEVICES] = []
  time_to_converge[N_DEVICES] = []
  for t in range(1,test_case_num+1):

    #reading data
    network_data = {}
    G = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/graph.gml",True)
    network_data['tx_time'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/tx_time.txt", False)
    network_data['max_edge_data_rate'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_edge_data_rate.txt",False)
    network_data['cst'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/cst.txt",False)
    network_data['max_flow_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_flow_delays.txt",False)
    network_data['edge_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/edge_delays.txt",False)
    network_data['max_edge_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_edge_delays.txt",False)
    network_data['data_rate'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/data_rate.txt",False)
    network_data['eh_time'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/eh_time.txt",False)
    network_data['init_res_energy'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/init_res_energy.txt",False)
    edge_nodes = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/edge_nodes.txt",False)


    #Get 2D coordinates of all the devices
    coordinates = {i:G.nodes()[i]['pos'] for i in G.nodes}

    #creating a list of bidirectional edges
    edge_list = []
    for u,v in G.edges:
      edge_list.append((u,v))
      edge_list.append((v,u))
    if expected_ans[N_DEVICES][t] == -1:
      invalid_graph = invalid_graph + 1
      continue
    optimal_cost, optimal_path, converge_iter, converge_time = PSO_Algorithm(edge_nodes, G, network_data)
    optimal_cost = round(optimal_cost, 2)
    print(f"num_nodes = {N_DEVICES}, test_case = {t}, actual_cost = {optimal_cost}, optimal_cost = {expected_ans[N_DEVICES][t]}")

    time_to_converge[N_DEVICES].append(converge_time)
    iterations_to_converge[N_DEVICES].append(converge_iter)
    results[N_DEVICES].append([optimal_cost,expected_ans[N_DEVICES][t]])
    percent_gap[N_DEVICES].append(((optimal_cost-expected_ans[N_DEVICES][t])/expected_ans[N_DEVICES][t])*100)
    total_tests = total_tests + 1
    if ((optimal_cost-expected_ans[N_DEVICES][t])/expected_ans[N_DEVICES][t])*100 <= 4:
      nearly_correct = nearly_correct + 1
    if optimal_cost == expected_ans[N_DEVICES][t]:
      correct_solution = correct_solution + 1
  
  #calculating route optimality ratio
  if total_tests == 0:
    continue
  invalid_tests[N_DEVICES] = invalid_graph
  near_route_optimality_ratio[N_DEVICES] = nearly_correct/total_tests
  near_route_failure_ratio[N_DEVICES] = (total_tests-nearly_correct)/total_tests
  route_optimality_ratio[N_DEVICES] = correct_solution/total_tests
  route_failure_ratio[N_DEVICES] = (total_tests-correct_solution)/total_tests

In [ ]:
#saving results
def save_file(file_name, results):
  with open("/content/gdrive/MyDrive/Networks/" + file_name, 'wb') as file:
    pickle.dump(results, file)
    file.close

In [ ]:
save_file("PSO_ad2_route_optimality_ratio.pkl", route_optimality_ratio)
save_file("PSO_ad2_route_failure_ratio.pkl", route_failure_ratio)
save_file("PSO_ad2_results.pkl", results)
save_file("PSO_ad2_percent_gap.pkl", percent_gap)
save_file("PSO_ad2_iterations_to_converge.pkl", iterations_to_converge)
save_file("PSO_ad2_time_to_converge.pkl", time_to_converge)